In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder

from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import d3
import bokeh.models as bmo
import bokeh.plotting as bpl
output_notebook()

Loading BokehJS ...

In [2]:
df = gpd.read_file('/Users/dianaow/Documents/singapore_stories/parliament/data/electoral-boundary-2020.json')
#Find the center point
df['Center_point'] = df['geometry'].centroid
#Extract lat and lon from the centerpoint
df["lat"] = df.Center_point.map(lambda p: p.x)
df["long"] = df.Center_point.map(lambda p: p.y)

/Users/dianaow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
df.head()

,Name,description,altitudeMode,id,FID,ED_DESC,Field_1,geometry,Center_point,lat,long
0,TANJONG PAGAR,None,clampToGround,TP,2,TANJONG PAGAR,TANJONG PAGAR,"MULTIPOLYGON (((103.84580 1.26163, 103.84640 1...",POINT (103.82306 1.30169),103.823059,1.301685
1,JALAN BESAR,None,clampToGround,JB,3,JALAN BESAR,JALAN BESAR,"MULTIPOLYGON (((103.87383 1.28217, 103.87385 1...",POINT (103.85724 1.30020),103.857243,1.300203
2,PIONEER,None,clampToGround,PI,5,PIONEER,PIONEER,"MULTIPOLYGON (((103.70830 1.34003, 103.70895 1...",POINT (103.70086 1.33820),103.700861,1.338201
3,YUHUA,None,clampToGround,YH,7,YUHUA,YUHUA,"MULTIPOLYGON (((103.73730 1.34596, 103.73681 1...",POINT (103.73820 1.33945),103.738196,1.339454
4,MARYMOUNT,None,clampToGround,MR,12,MARYMOUNT,MARYMOUNT,"MULTIPOLYGON (((103.85226 1.36033, 103.85194 1...",POINT (103.84275 1.35775),103.842752,1.357749


In [4]:
df_ward = gpd.read_file('/Users/dianaow/Documents/singapore_stories/parliament/data/GE2020.json')
#Find the center point
df_ward['Center_point'] = df_ward['geometry'].centroid
#Extract lat and lon from the centerpoint
df_ward["lat"] = df_ward.Center_point.map(lambda p: p.x)
df_ward["long"] = df_ward.Center_point.map(lambda p: p.y)

/Users/dianaow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
df_ward.head()

,Name,Description,geometry,Center_point,lat,long
0,MOUNTBATTEN SMC,GRC/SMC: Single Member Constituency<br>Member ...,"POLYGON Z ((103.90455 1.29702 0.00000, 103.903...",POINT (103.88395 1.29795),103.883951,1.297949
1,Marine Parade_3,GRC/SMC: MARINE PARADE GRC<br>Member of Parlia...,"POLYGON Z ((103.91000 1.30547 0.00000, 103.908...",POINT (103.90716 1.30685),103.907155,1.306847
2,Geylang Serai_2,GRC/SMC: MARINE PARADE GRC<br>Member of Parlia...,"POLYGON Z ((103.89621 1.33776 0.00000, 103.895...",POINT (103.89268 1.32820),103.892684,1.328197
3,Braddell Heights_1,GRC/SMC: MARINE PARADE GRC<br>Member of Parlia...,"POLYGON Z ((103.88018 1.34885 0.00000, 103.879...",POINT (103.87097 1.34847),103.870966,1.348472
4,MACPHERSON SMC,GRC/SMC: Single Member Constituency<br>Member ...,"POLYGON Z ((103.88910 1.33247 0.00000, 103.888...",POINT (103.88609 1.32399),103.886086,1.323992


In [6]:
wards = pd.read_csv("/Users/dianaow/Documents/singapore_stories/parliament/data/Singapore Members of Parliament - Wards.csv")
constituencies = pd.read_csv("/Users/dianaow/Documents/singapore_stories/parliament/data/Singapore Members of Parliament - Consituencies.csv")
designation = pd.read_csv("/Users/dianaow/Documents/singapore_stories/parliament/data/Singapore Members of Parliament - 14th_Parliament.csv")

In [7]:
constituencies['Division'] = constituencies['Division'].str.upper() 
constituencies = pd.merge(constituencies, df.rename(columns={"Name": 'Division'})\
                          [['Division', 'lat', 'long']], on='Division', how='left')

constituencies = pd.merge(constituencies, wards, on='Name', how='left')
constituencies = pd.merge(constituencies, df_ward.rename(columns={"Name": 'Ward', 'lat': 'lat_ward', 'long': 'lon_ward'})\
                          [['Ward', 'lat_ward', 'lon_ward']], on='Ward', how='left')

constituencies['Ward'] = constituencies['Ward'].apply(lambda x: x.split('_')[0])
constituencies['newName'] = constituencies['Name'].apply(lambda x : x.replace(' ', '_'))

constituencies = pd.merge(constituencies, designation.rename(columns={'Type': 'Title'}), on='Name', how='left')

In [8]:
constituencies['First_Occupation'] = constituencies['First_Occupation'].apply(lambda x: x.strip())
constituencies['Previous_Occupation'] = constituencies['Previous_Occupation'].apply(lambda x: x.strip())

In [9]:
constituencies['Title_1'] = constituencies['Title_1'].replace('Deputy Prime Minister', 'Senior Minister')
constituencies['Title_1'] = constituencies['Title_1'].replace('Minister i/c of Social Services Integration', 'Minister')

constituencies['Title_2'] = constituencies['Title_1'].replace('Deputy Prime Minister', 'Senior Minister')
constituencies['Title_2'] = constituencies['Title_1'].replace('Senior Minister of State', 'Minister of State')
constituencies['Title_2'] = constituencies['Title_1'].replace('Senior Parliamentary Secretary', 'Parliamentary Secretaries')

In [10]:
for i,row in constituencies.iterrows():
    if(constituencies.loc[i, "Age"] <= 35):
        constituencies.loc[i, "Age_group"] = 'Less than 35'
    elif((constituencies.loc[i, "Age"] > 35) & (constituencies.loc[i, "Age"] <= 40)):
        constituencies.loc[i, "Age_group"] = '35 - 40'
    elif((constituencies.loc[i, "Age"] > 40) & (constituencies.loc[i, "Age"] <= 45)):
        constituencies.loc[i, "Age_group"] = '40 - 45'
    elif((constituencies.loc[i, "Age"] > 45) & (constituencies.loc[i, "Age"] <= 50)):
        constituencies.loc[i, "Age_group"] = '45 - 50'
    elif((constituencies.loc[i, "Age"] > 50) & (constituencies.loc[i, "Age"] <= 55)):
        constituencies.loc[i, "Age_group"] = '50 - 55'
    elif((constituencies.loc[i, "Age"] > 55) & (constituencies.loc[i, "Age"] <= 60)):
        constituencies.loc[i, "Age_group"] = '55 - 60'
    elif((constituencies.loc[i, "Age"] > 60) & (constituencies.loc[i, "Age"] <= 65)):
        constituencies.loc[i, "Age_group"] = '60 - 65'
    elif(constituencies.loc[i, "Age"] > 65):
        constituencies.loc[i, "Age_group"] = 'More than 65'

In [11]:
constituencies.columns

Index(['Division', 'Type', 'Name', 'lat', 'long', 'Ward', 'lat_ward',
       'lon_ward', 'newName', 'Party', 'Age', 'Years_In_Politics', 'Ethnicity',
       'Gender', 'Grassroots_Experience', 'First_Occupation',
       'Previous_Occupation', 'Previous/Current_Role', 'Previous_Employer',
       'Title', 'Parliament', 'Type.1', 'Ministry_1', 'Ministry_2',
       'Ministry_3', 'Title_1', 'Title_2', 'Title_3', 'Defence',
       'Foreign Affairs', 'Health', 'Transport',
       'Sustainability and the Environment', 'Social & Family Development',
       'Communications & Information', 'Culture, Community and Youth',
       'Education', 'Trade and Industry', 'Manpower', 'Finance', 'PMO', 'Law',
       'National Development', 'Home Affairs', 'Age_group'],
      dtype='object')

In [12]:
constituencies.isnull().sum()

Division                               0
Type                                   1
Name                                   0
lat                                    0
long                                   0
Ward                                   0
lat_ward                               0
lon_ward                               0
newName                                0
Party                                  0
Age                                    0
Years_In_Politics                      0
Ethnicity                              0
Gender                                 0
Grassroots_Experience                  0
First_Occupation                       0
Previous_Occupation                    0
Previous/Current_Role                  2
Previous_Employer                      4
Title                                  0
Parliament                             0
Type.1                                 0
Ministry_1                            56
Ministry_2                            69
Ministry_3      

In [50]:
X_subset = constituencies[['Age_group', 'Years_In_Politics', 'Ethnicity', 'Gender', 'Title_1', \
                           'First_Occupation', 'Previous_Occupation']]

#X_subset = constituencies[['Age_group', 'Years_In_Politics', 'Ethnicity', 'Gender', 'Title_1', \
                           #'First_Occupation', 'Previous_Occupation', 'Defence', 'Foreign Affairs', 'Health', \
                           #'Transport', 'Sustainability and the Environment', 'Social & Family Development', \
                           #'Communications & Information', 'Culture, Community and Youth', 'Education', \
                           #'Trade and Industry', 'Manpower', 'Finance', 'PMO', 'Law', 'National Development', 'Home Affairs' ]]

In [51]:
enc = OneHotEncoder(handle_unknown='ignore')
X_enc = enc.fit_transform(X_subset).toarray()
enc_cols = enc.get_feature_names(X_subset.columns)
df_X = pd.DataFrame(X_enc, columns=enc_cols)

In [52]:
from sklearn.manifold import TSNE

RS = 123
tsne_features = TSNE(random_state=RS, perplexity=50.0, n_iter=5000, learning_rate=10).fit_transform(X_enc)

# Make X, Y columns 
constituencies['X'] = tsne_features[:, 0]
constituencies['Y'] = tsne_features[:, 1]

In [48]:
constituencies.columns

Index(['Division', 'Type', 'Name', 'lat', 'long', 'Ward', 'lat_ward',
       'lon_ward', 'newName', 'Party', 'Age', 'Years_In_Politics', 'Ethnicity',
       'Gender', 'Grassroots_Experience', 'First_Occupation',
       'Previous_Occupation', 'Previous/Current_Role', 'Previous_Employer',
       'Title', 'Parliament', 'Type.1', 'Ministry_1', 'Ministry_2',
       'Ministry_3', 'Title_1', 'Title_2', 'Title_3', 'Defence',
       'Foreign Affairs', 'Health', 'Transport',
       'Sustainability and the Environment', 'Social & Family Development',
       'Communications & Information', 'Culture, Community and Youth',
       'Education', 'Trade and Industry', 'Manpower', 'Finance', 'PMO', 'Law',
       'National Development', 'Home Affairs', 'Age_group', 'X', 'Y', 'color',
       'color1', 'color2', 'color4', 'color3', 'X1', 'Y1', 'code'],
      dtype='object')

In [17]:
for i,row in constituencies.iterrows():
    if(constituencies.loc[i, "Previous_Occupation"] == 'Public Service (Labour)'):
        constituencies.loc[i, "color"] = 'rgb(199,21,133)'
    elif(constituencies.loc[i, "Previous_Occupation"] == 'IT/Engineering'):
        constituencies.loc[i, "color"] = 'rgb(139,0,0)'
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Infrastructure (Transport/Telecomms/Real Estate)'):
        constituencies.loc[i, "color"] = 'rgb(255,69,0)'
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Public Service (Multiple Stat Boards)'):
        constituencies.loc[i, "color"] = 'rgb(255,215,0)'
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Social Enterprise/Charity'):
        constituencies.loc[i, "color"] = 'rgb(0,100,0)'
    elif(constituencies.loc[i, "Previous_Occupation"] == 'SAF/SPF/SCDF'):
        constituencies.loc[i, "color"] = 'rgb(50,205,50)'
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Law'):
        constituencies.loc[i, "color"] = 'rgb(0,128,128)'
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Education'):
        constituencies.loc[i, "color"] = 'rgb(255,228,225)' 
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Finance'):
        constituencies.loc[i, "color"] = 'rgb(47,79,79)' 
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Entrepreneur'):
        constituencies.loc[i, "color"] = 'rgb(0,0,0)' 
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Public Service (Finance)'):
        constituencies.loc[i, "color"] = 'rgb(255, 105, 180)' 
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Biz Dev'):
        constituencies.loc[i, "color"] = 'rgb(255,0,0)' 
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Specialist - Healthcare'):
        constituencies.loc[i, "color"] = 'rgb(0,255,0)' 
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Public Service (Social/Healthcare)'):
        constituencies.loc[i, "color"] = 'rgb(0,0,255)' 
    elif(constituencies.loc[i, "Previous_Occupation"] == 'Public Service / Govt-linked (Biz Dev)'):
        constituencies.loc[i, "color"] = 'rgb(188,143,143)' 
    else:
        constituencies.loc[i, "color"] = 'rgb(128,0,128)' 

In [18]:
for i,row in constituencies.iterrows():
    if(constituencies.loc[i, 'Grassroots_Experience'] == 'Yes'):
        constituencies.loc[i, "color1"] = 'rgb(199,21,133)'
    elif(constituencies.loc[i, 'Grassroots_Experience'] == 'No'):
        constituencies.loc[i, "color1"] = 'rgb(139,0,0)'

In [19]:
for i,row in constituencies.iterrows():
    if(constituencies.loc[i, 'Gender'] == 'Male'):
        constituencies.loc[i, "color2"] = 'rgb(199,21,133)'
    elif(constituencies.loc[i, 'Gender'] == 'Female'):
        constituencies.loc[i, "color2"] = 'rgb(139,0,0)'

In [20]:
for i,row in constituencies.iterrows():
    if(constituencies.loc[i, 'Party'] == "People's Action Party"):
        constituencies.loc[i, "color4"] = 'rgb(199,21,133)'
    elif(constituencies.loc[i, 'Party'] == "Workers' Party"):
        constituencies.loc[i, "color4"] = 'rgb(139,0,0)'

In [21]:
for i,row in constituencies.iterrows():
    if(constituencies.loc[i, "Ethnicity"] == 'Chinese'):
        constituencies.loc[i, "color3"] = 'rgb(0,0,255)'
    elif(constituencies.loc[i, "Ethnicity"] == 'Malay'):
        constituencies.loc[i, "color3"] = 'rgb(0,255,0)'
    elif(constituencies.loc[i, "Ethnicity"] == 'Indian'):
        constituencies.loc[i, "color3"] = 'rgb(255,0,0)'
    elif(constituencies.loc[i, "Ethnicity"] == 'Eurasian'):
        constituencies.loc[i, "color3"] = 'rgb(0,255,255)'

In [22]:
# Make a source and a scatter plot  
source = ColumnDataSource(constituencies)

plot = figure(x_axis_label = 'T-SNE 1', 
              y_axis_label = 'T-SNE 2', 
              width = 500, height = 400)


plot.scatter(
    "X", "Y", 
    source = source, 
    size = 10, 
    color='color', 
    alpha = .5)

GlyphRenderer(id='1040', ...)

In [23]:
# Create a HoverTool object
hover = HoverTool(tooltips = [('Name', '@Name')])
plot.add_tools(hover)

In [24]:
# Plot the map
show(plot)

In [25]:
# Print the ingredients of two similar cosmetics
p1 = constituencies[constituencies['Name'] == "Jessica Tan"]
p2 = constituencies[constituencies['Name'] == "Sylvia Lim"]

# Display each item's data and ingredients
display(p1)
display(p2)

,Division,Type,Name,lat,long,Ward,lat_ward,lon_ward,newName,Party,...,National Development,Home Affairs,Age_group,X,Y,color,color1,color2,color4,color3
37,EAST COAST,GRC,Jessica Tan,104.003926,1.369603,Changi Simei,103.952592,1.340778,Jessica_Tan,People's Action Party,...,0,0,50 - 55,-3.49133,-4.15283,"rgb(128,0,128)","rgb(139,0,0)","rgb(139,0,0)","rgb(199,21,133)","rgb(0,0,255)"


,Division,Type,Name,lat,long,Ward,lat_ward,lon_ward,newName,Party,...,National Development,Home Affairs,Age_group,X,Y,color,color1,color2,color4,color3
77,ALJUNIED,GRC,Sylvia Lim,103.90293,1.355507,Paya Lebar,103.88675,1.356103,Sylvia_Lim,Workers' Party,...,0,0,50 - 55,-2.240193,-4.71964,"rgb(255,228,225)","rgb(139,0,0)","rgb(139,0,0)","rgb(139,0,0)","rgb(0,0,255)"


In [26]:
constituencies[constituencies['Name'] == "Teo Chee Hean"]['Age_group']

82    60 - 65
Name: Age_group, dtype: object

In [65]:
list_1 = ['Age_group', 'Ethnicity', 'Gender', 'First_Occupation', 'Previous_Occupation', 'Years_In_Politics']
X_subset1 = constituencies[list_1]

enc = OneHotEncoder(handle_unknown='ignore')
X_enc1 = enc.fit_transform(X_subset1).toarray()
enc_cols1 = enc.get_feature_names(X_subset1.columns)
df_X1 = pd.DataFrame(X_enc1, columns=enc_cols1)

prev_cols = ['Previous_Occupation_Biz Dev', 'Previous_Occupation_Education', 'Previous_Occupation_Entrepreneur', \
'Previous_Occupation_Finance', 'Previous_Occupation_IT/Engineering', 'Previous_Occupation_Infrastructure (Transport/Telecomms/Real Estate)', \
'Previous_Occupation_Law', 'Previous_Occupation_Public Service (Finance)', 'Previous_Occupation_Public Service (Labour)', \
'Previous_Occupation_Public Service (Multiple Stat Boards)', 'Previous_Occupation_Public Service (Multiple Stat Boards) ', \
'Previous_Occupation_Public Service (Social/Healthcare)', 'Previous_Occupation_Public Service (Social/Healthcare) ', \
'Previous_Occupation_Public Service / Govt-linked (Biz Dev)', 'Previous_Occupation_SAF/SPF/SCDF',\
'Previous_Occupation_Social Enterprise/Charity', 'Previous_Occupation_Specialist - Healthcare']

#for i in prev_cols:
    #df_X1[i + '_1'] = df_X1[i]
    
#X_enc1 = df_X1.to_numpy()

tsne_features1 = TSNE(random_state=RS,  perplexity=50.0, n_iter=5000, learning_rate=10).fit_transform(X_enc1)

# Make X, Y columns 
constituencies['X1'] = tsne_features1[:, 0]
constituencies['Y1'] = tsne_features1[:, 1]

In [66]:
# Make a source and a scatter plot  
source = ColumnDataSource(constituencies)

plot = figure(x_axis_label = 'T-SNE 1', 
              y_axis_label = 'T-SNE 2', 
              width = 500, height = 400)

plot.circle(x = "X1", 
    y = "Y1", 
    source = source, 
    size = 10, color = 'color', alpha = .5)

GlyphRenderer(id='1263', ...)

In [67]:
# Create a HoverTool object
hover = HoverTool(tooltips = [('Name', '@Name')])
plot.add_tools(hover)

In [68]:
# Plot the map
show(plot)

In [31]:
# Print the ingredients of two similar MPs
p1 = constituencies[constituencies['Name'] == "Cheng Li Hui"][list_1]
p2 = constituencies[constituencies['Name'] == "Cheryl Chan"][list_1]

# Display each item's data and ingredients
display(p1)
display(p2)

,Age_group,Ethnicity,Gender,First_Occupation,Previous_Occupation,Years_In_Politics
8,40 - 45,Chinese,Female,IT/Engineering,IT/Engineering,5


,Age_group,Ethnicity,Gender,First_Occupation,Previous_Occupation,Years_In_Politics
9,40 - 45,Chinese,Female,IT/Engineering,IT/Engineering,5


In [53]:
constituencies.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/mps_constituencies.csv')

In [33]:
constituencies['Previous_Occupation'].unique()

array(['Public Service (Labour)', 'Other(HR/Management)',
       'Infrastructure (Transport/Telecomms/Real Estate)',
       'Public Service (Multiple Stat Boards)',
       'Social Enterprise/Charity', 'SAF/SPF/SCDF', 'IT/Engineering',
       'Law', 'Education', 'Finance',
       'Public Service (Social/Healthcare)', 'Entrepreneur',
       'Public Service (Finance)', 'Biz Dev', 'Specialist - Healthcare',
       'Public Service / Govt-linked (Biz Dev)'], dtype=object)

In [34]:
df = pd.read_csv("./data/Singapore Members of Parliament - Members_Parliament_Long.csv")

In [35]:
df.head()

,Name,Party,Ethnicity,Gender,Type,Parliament
0,A. Rahim Ishak,People's Action Party,Malay,Male,MP,1
1,A.P. Rajah,Independent,Malay,Male,MP,1
2,Ang Nam Piau,People's Action Party,Chinese,Male,MP,1
3,"Bani, S.T.",Barisan Sosialis,Indian,Male,MP,1
4,"Barker, E.W.",People's Action Party,Eurasian,Male,MP,1


In [36]:
df = df[df['Type'] == 'MP']

In [37]:
df_ethnicity = df.groupby(['Parliament', 'Ethnicity'])['Name'].count().reset_index()
df_ethnicity.rename(columns={'Name': 'value'}, inplace=True)
df_ethnicity_pt =  pd.pivot_table(df_ethnicity, index='Parliament', columns='Ethnicity', values='value', fill_value=0).reset_index(drop=True)
df_ethnicity_pt.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/ethnicity_timeline_PT.csv')
df_ethnicity.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/ethnicity_timeline.csv')

In [38]:
df_gender = df.groupby(['Parliament', 'Gender'])['Name'].count().reset_index()
df_gender.rename(columns={'Name': 'value'}, inplace=True)
df_gender_pt =  pd.pivot_table(df_gender, index='Parliament', columns='Gender', values='value', fill_value=0).reset_index()
df_gender_pt.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/gender_timeline_PT.csv')
df_gender.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/gender_timeline.csv')

In [61]:
df_party = df.groupby(['Parliament', 'Party'])['Name'].count().reset_index()
df_party.rename(columns={'Name': 'value'}, inplace=True)
df_party_pt =  pd.pivot_table(df_party, index='Parliament', columns='Party', values='value', fill_value=0).reset_index()
df_party_pt.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/party_timeline_PT.csv')

In [62]:
# Define the sorter
sorter = ["People's Action Party", "Workers' Party", 'Barisan Sosialis', 'Independent', \
 'Singapore Democratic Alliance','Singapore Democratic Party']

# Create the dictionary that defines the order for sorting
sorterIndex = dict(zip(sorter, range(len(sorter))))

# Generate a rank column that will be used to sort
# the dataframe numerically
df_party['Party_Rank'] = df_party['Party'].map(sorterIndex)

df_party.sort_values(['Parliament','Party_Rank'], ascending = True, inplace = True)

df_party.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/party_timeline.csv')

In [64]:
constituencies['code'] = constituencies['Defence'].astype(str) + constituencies['Foreign Affairs'].astype(str) + \
constituencies['Health'].astype(str) + constituencies['Transport'].astype(str) + \
constituencies['Sustainability and the Environment'].astype(str) + constituencies['Social & Family Development'].astype(str) + \
constituencies['Communications & Information'].astype(str) + constituencies['Culture, Community and Youth'].astype(str) + \
constituencies['Education'].astype(str) + constituencies['Trade and Industry'].astype(str) + \
constituencies['Manpower'].astype(str) + constituencies['Finance'].astype(str) + \
constituencies['PMO'].astype(str) + constituencies['Law'].astype(str) + \
constituencies['National Development'].astype(str) + constituencies['Home Affairs'].astype(str)